In [33]:
#!pip install numpy

In [34]:
import numpy as np
from scipy.optimize import fsolve

In [ ]:
class Dynamics_SGLF():
    def __init__(self, a, t0, tfinal, nsteps, z0, M, e):
        self.a = a
        self.t0 = t0
        self.tfinal = tfinal
        self.nsteps = nsteps #количество шагов по времени
        self.n0step = 0
        self.z0 = z0
        self.times = np.linspace(self.t0, self.tfinal, self.nsteps)
        self.e = e
        self.M = M

    @staticmethod
    def Sun_position_SSB(M, R, M_sun):
        '''
        R - массив, содержащий радиус-векторы от Солнца до планет в км?
        M - массив, содержащий массы планет в кг?
        M: np.array
        
        '''
        r_sun = np.zeros_like(R[0])
        for i in range(4):
            r_sun += -M[i] * R[i] / (M_sun + M[i])
        return r_sun

    def barycenter_position_SSB(self, t):
        '''
        Вычисл положения барицентра экзосистемы в SSB
        '''
        # переход в сферич координаты
        phi = 90. + 50. * 1e-6 * (t-self.t0) # в град
        psi = 0. + 50. * 1e-6 * (t-self.t0) # в град
        n_bc = np.array([[np.cos(psi)*np.cos(phi)],
                         [np.sin(psi)*np.cos(phi)],
                         [np.sin(phi)]])
        #r_bc = z0 * n_bc
        return n_bc
    
    def delta(r_p_bc, r_sun):
        return r_p_bc - r_sun
    
    def focal_line_direction(self, n_bc, delta):
        n0 = - n_bc - (delta - np.dot(delta, n_bc) * n_bc)/self.z0
        return n0

    def basis_FCS(self, n0):
        g3 = n0 / np.linalg.norm(n0)
        g1 = (self.a - np.dot(self.a,g3) * g3) / np.linalg.norm(self.a - np.dot(self.a,g3) * g3)
        g2 = np.cross(g3, g1)
        S_matrix = np.array([g1, g2, g3])
        return S_matrix
    
    def d_delta_dt(self, r_p_bc, r_sun):
        #Вычисляем производную \delta'(t) = d(r_p_bc - r_sun)/dt
        dr_p_bc = np.gradient(r_p_bc, self.times, axis=0)
        dr_sun = np.gradient(r_sun, self.times, axis=0)
        d_delta_dt = dr_p_bc - dr_sun
        d2_delta_dt = np.gradient(dr_p_bc, self.times, axis=0) - np.gradient(dr_sun, self.times, axis=0)
        return d_delta_dt, d2_delta_dt
    
    def d_n0_dt(self, n_bc, r_p_bc, r_sun):
        delta = self.delta(r_p_bc, r_sun)
        d_delta_dt = self.d_delta_dt(r_p_bc, r_sun)[0]
        d2_delta_dt = self.d_delta_dt(r_p_bc, r_sun)[1]

        d_n_bc_dt = np.gradient(n_bc, self.times, axis=0)
        d2_n_bc_dt = np.gradient(d_n_bc_dt, self.times, axis=0)

        delta_normal = delta - np.dot(n_bc, delta) * n_bc
        d_delta_normal_dt = d_delta_dt - np.dot(d_n_bc_dt, delta) * n_bc - np.dot(n_bc, d_delta_dt) * n_bc - np.dot(n_bc, delta) * d_n_bc_dt
        d2_delta_normal_dt = (
                    d2_delta_dt 
                    - np.dot(d2_n_bc_dt, delta) * n_bc
                    - np.dot(n_bc, d2_delta_dt) * n_bc
                    - np.dot(n_bc, delta) * d2_n_bc_dt
                    - 2 * (
                        np.dot(d_n_bc_dt, d_delta_dt) * n_bc 
                        + np.dot(d_n_bc_dt, delta) * d_n_bc_dt 
                        + np.dot(n_bc, d_delta_dt) * d_n_bc_dt
                    )
                )
        
        d_n0_dt = - d_n_bc_dt - d_delta_normal_dt / self.z0
        d2_n0_dt = - d2_n_bc_dt - d2_delta_normal_dt / self.z0
        return d_n0_dt, d2_n0_dt
    
    def d_S_dt(self, g1, g2, g3, n_bc, r_p_bc, r_sun):
        '''
        Первая и вторая производная матрицы S , составленной из базисных векторов g1, g2, g3 системы FCS
        
        '''
        d_n0_dt = self.d_n0_dt(n_bc, r_p_bc, r_sun)[0]
        d2_n0_dt = self.d_n0_dt(n_bc, r_p_bc, r_sun)[1]
        delta = self.delta(r_p_bc, r_sun)
        n0 = self.focal_line_direction(n_bc, delta)
        d_g3_dt = (d_n0_dt - np.dot(g3, d_n0_dt)) / np.linalg.norm(n0)
        d2_g3_dt = (d2_n0_dt - np.dot(g3, d2_n0_dt) * g3 - 2 * np.dot(g3, d_n0_dt) * d_g3_dt 
                    - np.dot(d_g3_dt, d_n0_dt) * g3 ) / np.linalg.norm(n0)

        a_normal = - np.dot(self.a,g3) * g3 + self.a
        d_a_normal_dt =  - np.dot(self.a,d_g3_dt) * g3 - np.dot(self.a,g3) * d_g3_dt
        d2_a_normal_dt = - np.dot(self.a,d2_g3_dt) * g3 - 2 * np.dot(self.a,d_g3_dt) * d_g3_dt - np.dot(self.a,g3) * d2_g3_dt
        
        d_g1_dt = (a_normal - np.dot(g1, d_a_normal_dt) * g1) / np.linalg.norm(a_normal)
        d2_g1_dt = (d2_a_normal_dt - np.dot(g1, d2_a_normal_dt) * g1 - 2 * np.dot(g1, d_a_normal_dt) * d_g1_dt 
                    - np.dot(d_g1_dt, d_a_normal_dt) * g1 ) / np.linalg.norm(a_normal)
        
        d_g2_dt = np.cross(d_g3_dt, g1) + np.cross(g3, d_g1_dt)
        d2_g2_dt = np.cross(d2_g3_dt, g1) + 2 * np.cross(d_g3_dt, d_g1_dt) + np.cross(g3, d2_g1_dt) 

        dS_dt = np.array([d_g1_dt, d_g2_dt, d_g3_dt])
        d2S_dt = np.array([d2_g1_dt, d2_g2_dt, d2_g3_dt])
        return dS_dt, d2S_dt

    def kepler_problem(self, E):
        return E - self.e * np.sin(E) - self.M
    
    def kepler_solve(self):
        E0 = self.M  # начальное приближение
        E = fsolve(self.kepler_problem, E0, args=(self.M, self.e))[0] # находим корень f(x) = 0 
        return E

    def r_p_bc_kepler(self, t, a1):
        '''функция для вычисления радиус-вектора r
         a1 - большая полуось!
           '''
        # Вычисляем среднюю аномалию M
        dt = (t - self.t0).days / 365.25  # Время в годах
        self.M = 2 * np.pi * (dt % 1) 
        E = self.kepler_solve()

        # Истинная аномалия θ
        theta = 2 * np.arctan(np.sqrt((1 + self.e) / (1 - self.e)) * np.tan(E / 2))

        # Радиус-вектор r
        r_p_bc = a1 * (1 - self.e**2) / (1 + self.e * np.cos(theta))

        return r_p_bc
    

In [36]:
# M = np.array([1., 1., 2., 3.])
# R = np.array([[1., 1., 1.],
#               [2.,1.,1.],
#               [1.,1.,3.],
#               [3., 4., 0.]])

# x = Dynamics_SGLF(a , t0 = 0., tfinal=10., nsteps=10, z0=10.)

# x.position_Sun_SSB(M, R, M_sun)

In [37]:
# a= np.array([1., 0., 0.])
# x = Dynamics_SGLF(a , t0 = 0., tfinal=10., nsteps=10, z0=10.)
# n0 = np.array([0., 1., 1.])
# x.basis_FCS(n0)